# Homework 1

## 1.1

Introspection might give a little insight into the congnitive processes, but I don't think it is valuable for modelling the mind. If you really wanted to model the human mind, monitoring it through experiments or through external observation would seem to be more powerful. The problem with instrospection is that it is subjective, despite the attempts for the practitioner to be unbiased in their analysis. It seems that humans tend to overestimate their thought processes/intelligence, when it really isn't all that complicated. Rather, it's the simple, unconscious subprocesses that are impressive aspects of the brain.

## 1.2

I formulated the problem by representing the path of the travelling salesman as an ordered array of locations that loops around to the start. The locations are represented using their (x, y) coordinates, and the distances between two locations is calculated using the straight distance between the two points. The value function calculates the total distance of the path using these metrics (and makes it negative for max value processing). The actions are unique combinations of two locations that get swapped in the result function. The goal test always returns false because there is no known solution.

In [106]:
from search import Problem, hill_climbing, simulated_annealing, exp_schedule
import itertools
import math

class TSP(Problem):
    def __init__(self, locations, initial):
        self.locations = locations
        self.initial = initial
     
    def actions(self, state): 
        # Get all the unique pair combinations of the state
        return list([c for c in itertools.combinations(state, 2)])
    
    def result(self, state, move):
        new_state = state[:]
        # Swap the position of the two locations
        i, j = new_state.index(move[0]), new_state.index(move[1])
        new_state[i], new_state[j] = new_state[j], new_state[i]
        return new_state
    
    def value(self, state):
        value = 0
        # Calculate the total distance traveled given the state
        for i in range(len(state)):
            # Get the (x, y) coordinates of the current and the next location in the path
            a, b = self.locations[state[i]], self.locations[state[(i + 1) % len(state)]]
            value += math.sqrt((b[0] - a[0])**2 + (b[1] - a[1])**2)
        return -value

I simply used the lab code from the Romania problem as my city formulation. I simply use the (x, y) coordinates of locations, but the problem could easily be switched to use a dictionary of distances between each city. The initial state is generated by taking the keys of the dictionary and randomly shuffling them so we start with a random path.

In [108]:
import random
locations = dict(
    arad=(91, 492), 
    bucharest=(400, 327), 
    craiova=(253, 288), 
    dobreta=(165, 299),
    eforie=(562, 293), 
    fagaras=(305, 449), 
    giurgiu=(375, 270), 
    hirsova=(534, 350),
    iasi=(473, 506), 
    lugoj=(165, 379), 
    mehadia=(168, 339), 
    neamt=(406, 537),
    oradea=(131, 571), 
    pitesti=(320, 368), 
    rimnicuvilcea=(233, 410), 
    sibiu=(207, 457),
    timisoara=(94, 410), 
    urziceni=(456, 350), 
    vaslui=(509, 444), 
    zerind=(108, 531))

initial = list(locations.keys())
random.shuffle(initial)
p = TSP(locations, initial)

hill_solution = hill_climbing(p)
print('Hill-climbing:')
print('\tSolution: ' + str(hill_solution))
print('\tValue:    ' + str(-p.value(hill_solution)))


annealing_solution = simulated_annealing(
    p,
    exp_schedule(k=20, lam=0.005, limit=15000)
)
print('Simulated annealing:')
print('\tSolution: ' + str(annealing_solution))
print('\tValue:    ' + str(-p.value(annealing_solution)))

Hill-climbing:
	Solution: ['eforie', 'hirsova', 'vaslui', 'iasi', 'neamt', 'fagaras', 'pitesti', 'craiova', 'dobreta', 'mehadia', 'oradea', 'zerind', 'arad', 'timisoara', 'lugoj', 'sibiu', 'rimnicuvilcea', 'giurgiu', 'bucharest', 'urziceni']
	Value:    1823.7643039544705
Simulated annealing:
	Solution: ['bucharest', 'pitesti', 'fagaras', 'neamt', 'iasi', 'vaslui', 'hirsova', 'eforie', 'urziceni', 'rimnicuvilcea', 'sibiu', 'oradea', 'zerind', 'arad', 'timisoara', 'lugoj', 'mehadia', 'dobreta', 'craiova', 'giurgiu']
	Value:    1757.2138373744772


Simulated annealing consistently performs much better than the hill-climbing. But if random restarts were implemented, the gap in perfomance would shrink a little. Hill-climbing is so dependent on the initial state that it usually ends up at a poor local maximum. But simulated annealing has the potential to explore more areas than just it's starting state. 

## 1.3

I structured CSP schedule problem by having the classes as the variables, and the domain being all combinations of classrooms, professors, and times. The user can give a schedule defined in a dictionary format that also includes the assignment of professor to class. The Schedule functions takes the dictionary schedule, parses it, and then returns a CSP class with the specified parameters. The constraint function ensures that: 
    1. professors are assigned to their classes
    2. if the classes are at the same time, they are in a different room
    3. if it's the same professor, they are teaching at different times

In [109]:
from csp import CSP
import itertools

"""
example_schedule = dict(
    classrooms='nh253 sb383'.split(),
    times='mwf900 mwf1000 mwf1100'.split(),
    classes=dict(
        cs108='norman',
        cs112='adams',
        cs212='schuurman',
        cs262='vanderlinden',
        cs344='vanderlinden',
    )
)
"""

def Schedule(schedule):
    # Get the variables from the schedule
    variables = list(schedule['classes'].keys())
    
    professors = list(set(list(schedule['classes'].values())))
    # All possible combinations of rooms, professors, and times
    possible_values = list(itertools.product(schedule['classrooms'], professors, schedule['times']))
    # Map possible values to the domain of each variable
    domains = dict(map(lambda x: (x, possible_values[:]), variables))
    
    # For each variable, get a list of the other classes that doesn't include the variable
    neighbors = dict(map(lambda i: (i, list(filter(lambda j: j != i, variables))), variables))
    
    def schedule_constraint(A, a, B, b):
        class_location = professor_time = professor_class = True
        # Professor is actually teaching their assigned class
        professor_class = schedule['classes'][A] == a[1] and schedule['classes'][B] == b[1]
        # If the time is the same, classroom has to be different
        if (a[2] == b[2]):
            class_location = a[0] != b[0]
        # If the professor is the same, the time has to be different
        if (a[1] == b[1]):
            professor_time = a[2] != b[2]
            
        return professor_class and professor_time and class_location
    
    return CSP(variables, domains, neighbors, schedule_constraint)


I structured the schedule object in the simplest way possible without repeated data and just let the Schedule function to parse everything. The backtracking search works fine on these size of problems, even with the larger schedule, the extra optimizations aren't really necessary for performance. The min conflicts algorithm also works on the problems. 

In [111]:
from csp import backtracking_search, mrv, forward_checking, AC3, min_conflicts
example_schedule = dict(
    classrooms='nh253 sb383'.split(),
    times='mwf900 mwf1000 mwf1100'.split(),
    classes=dict(
        cs108='norman',
        cs112='adams',
        cs212='schuurman',
        cs262='vanderlinden',
        cs344='vanderlinden',
    )
)

def print_solution(schedule):
    for key in schedule:
        print(key, ": ", schedule[key])

print("Example Schedule")
solution = backtracking_search(Schedule(example_schedule))
if (solution):
    print_solution(solution)
else:
    print('Failed')
    
large_schedule = dict(
    classrooms='nh253 sb383 nh200'.split(),
    times='mwf900 mwf1000 mwf1100 tth900 tth1000'.split(),
    classes=dict(
        cs100='norman',
        cs101='bailey',
        cs102='schuurman',
        cs103='vanderlinden',
        cs104='vanderlinden',
        cs105='vanderlinden',
        cs108='norman',
        cs112='adams',
        cs212='schuurman',
        cs262='vanderlinden',
        cs344='vanderlinden',
    )
)
print("Large Schedule")
#solution = min_conflicts(Schedule(large_schedule))
solution = backtracking_search(Schedule(large_schedule), select_unassigned_variable=mrv, inference=forward_checking)
if (solution):
    print_solution(solution)
else:
    print('Failed')

Example Schedule
cs108 :  ('nh253', 'norman', 'mwf900')
cs112 :  ('nh253', 'adams', 'mwf1000')
cs212 :  ('nh253', 'schuurman', 'mwf1100')
cs262 :  ('sb383', 'vanderlinden', 'mwf900')
cs344 :  ('sb383', 'vanderlinden', 'mwf1000')
Large Schedule
cs108 :  ('nh253', 'norman', 'mwf900')
cs100 :  ('nh253', 'norman', 'mwf1000')
cs105 :  ('nh253', 'vanderlinden', 'mwf1100')
cs262 :  ('nh253', 'vanderlinden', 'tth900')
cs103 :  ('nh253', 'vanderlinden', 'tth1000')
cs104 :  ('sb383', 'vanderlinden', 'mwf900')
cs344 :  ('sb383', 'vanderlinden', 'mwf1000')
cs112 :  ('sb383', 'adams', 'mwf1100')
cs212 :  ('sb383', 'schuurman', 'tth900')
cs102 :  ('sb383', 'schuurman', 'tth1000')
cs101 :  ('nh200', 'bailey', 'mwf900')
